## LISS4 Image Processing using XArray and Dask

This notebook shows how to pre-process scenes from the LISS4 sensor aboard ResourceSat2/2A.

Refere to blog [this article](https://spatialthoughts.com/2023/12/25/liss4-processing-xarray/) for a detailed guide.


In [1]:
!which python

/home/nischal/projects/cloudSnip/cloudSnip/processing/.venv/bin/python


In [2]:
import datetime
import ephem
import math
import os
import rioxarray as rxr
import xarray as xr
import zipfile

In [3]:
from dask.distributed import Client, progress
client = Client()  # set up local cluster on the machine

In [4]:
from pathlib import Path


def process_liss4_data(liss4_dir_zip, output_dir):
  for liss4_zip in liss4_dir_zip.glob("*.zip"):
    with zipfile.ZipFile(liss4_zip) as zf:
      # The LISS4 zip files contain a folder with all the data
      # Get the folder name
      foldername = [info.filename for info in zf.infolist() if info.is_dir()][0]
      # Extract all the data
      zf.extractall()
      
    metadata_filename = 'BAND_META.txt'
    metadata_filepath = os.path.join(foldername, metadata_filename)

    metadata = {}
    with open(metadata_filepath) as f:
      for line in f:
        key, value = line.split('=')
        metadata[key.strip()] = value.strip()

    scene_id = metadata['OTSProductID']

    b2_path = os.path.join(foldername, 'BAND2.tif')
    b3_path = os.path.join(foldername, 'BAND3.tif')
    b4_path = os.path.join(foldername, 'BAND4.tif')

    b2_ds = rxr.open_rasterio(b2_path, chunks=True)
    b3_ds = rxr.open_rasterio(b3_path, chunks=True)
    b4_ds = rxr.open_rasterio(b4_path, chunks=True)

    scene = xr.concat([b2_ds, b3_ds, b4_ds], dim='band').assign_coords(
        band=['BAND2', 'BAND3', 'BAND4'])

    scene = scene.where(scene != 0)
    scene.name = scene_id
    acq_date_str = metadata['DateOfPass']
    # Date is in the format 04-MAR-2023
    acq_date = datetime.datetime.strptime(acq_date_str, '%d-%b-%Y')

    sun_elevation_angle = metadata['SunElevationAtCenter']
    sun_zenith_angle = 90 - float(sun_elevation_angle)

    observer = ephem.Observer()
    observer.date = acq_date
    sun = ephem.Sun()
    sun.compute(observer)
    d = sun.earth_distance

    b2_sr = 53.0
    b3_sr = 47.0
    b4_sr = 31.5

    b2_esun = 181.89
    b3_esun = 156.96
    b4_esun = 110.48

    pi = math.pi
    sun_zenith_angle_rad = math.radians(sun_zenith_angle)

    b2_dn = scene.sel(band='BAND2')
    b3_dn = scene.sel(band='BAND3')
    b4_dn = scene.sel(band='BAND4')

    b2_rad = b2_dn*b2_sr/1024
    b3_rad = b3_dn*b3_sr/1024
    b4_rad = b4_dn*b4_sr/1024

    b2_ref = (pi*b2_rad*d*d)/(b2_esun*math.cos(sun_zenith_angle_rad))
    b3_ref = (pi*b3_rad*d*d)/(b3_esun*math.cos(sun_zenith_angle_rad))
    b4_ref = (pi*b4_rad*d*d)/(b4_esun*math.cos(sun_zenith_angle_rad))

    reflectance_bands = [b2_ref, b3_ref, b4_ref]
    scene_ref = xr.concat(reflectance_bands, dim='band').assign_coords(
        band=['BAND2', 'BAND3', 'BAND4']).chunk('auto')
    scene_ref.name = scene_id

    output_ds = scene_ref.to_dataset('band')


    output_file = output_dir / f'{scene_id}.tif'

    output_options = {
        'driver': 'COG',
        'compress': 'deflate',
        'num_threads': 'all_cpus',
        'windowed': False # set True if you run out of RAM
    }

    output_ds[['BAND2', 'BAND3', 'BAND4']].rio.to_raster(
        output_file, **output_options)


In [5]:
liss4_dir_zip = Path('/home/nischal/projects/cloudSnip/data/test_unprocessed/test_dir/TestData-Cloud-Shadow')
output_dir = Path('/home/nischal/projects/cloudSnip/data/unprocessed_data/val/img/')

process_liss4_data(liss4_dir_zip, output_dir)

2025-07-06 14:15:47,041 - distributed.worker.memory - WARNING - Worker is at 86% memory usage. Pausing worker.  Process memory: 6.77 GiB -- Worker memory limit: 7.84 GiB
2025-07-06 14:15:48,144 - distributed.worker.memory - WARNING - Worker is at 64% memory usage. Resuming worker. Process memory: 5.05 GiB -- Worker memory limit: 7.84 GiB
2025-07-06 14:16:29,219 - distributed.worker.memory - WARNING - Worker is at 88% memory usage. Pausing worker.  Process memory: 6.93 GiB -- Worker memory limit: 7.84 GiB
2025-07-06 14:16:30,224 - distributed.worker.memory - WARNING - Worker is at 68% memory usage. Resuming worker. Process memory: 5.35 GiB -- Worker memory limit: 7.84 GiB


In [4]:
# if not liss4_zip:
#   liss4_zip = 'RAF20FEB2023032197010000064SSANSTUC00GTDC.zip'
#   data_url = 'https://storage.googleapis.com/spatialthoughts-public-data/liss4/'
#   url = data_url + liss4_zip
#   filename = os.path.basename(url)
#   if not os.path.exists(filename):
#       from urllib.request import urlretrieve
#       local, _ = urlretrieve(url, filename)
#       print('Downloaded demo scene: ' + local)